In [23]:
import pandas as pd
from glob import glob

def create_demand_distribution(excel_files, filters):
    # Combine data from all Excel sheets into a single DataFrame
    all_data = pd.DataFrame()
    for file in excel_files:
        df = pd.read_excel(file)  # Read each Excel file
        all_data = pd.concat([all_data, df], ignore_index=True)  # Append to the main DataFrame

    # Apply filters for year and college
    filtered_data = all_data[
        (all_data['YEAR'].isin(filters['YEAR'])) & 
        (all_data['COLLEGE'].isin(filters['COLLEGE']))
    ]

    # Group by bus stop and count the number of students
    demand_distribution = filtered_data.groupby('BUSTOP').size().to_dict()

    return demand_distribution

def create_location_coordinates(file_path):
    """
    Input: Relative path of the excel sheet which has the lat, long and location details
    Output: coordinates_dict
    {location: (lat, long)}
    """
    # Load the Excel file
    df = pd.read_excel(file_path)  # Replace with your file name

    # Create a dictionary where the key is 'Location' and the value is a tuple (Latitude, Longitude)
    coordinates_dict = df.set_index("Location")[["Latitude", "Longitude"]].apply(tuple, axis=1).to_dict()

    # Print the dictionary
    return coordinates_dict

def create_routes_list(route_list_file_path):
    # Read the Excel sheet
    df = pd.read_excel(route_list_file_path)

    # Assuming the columns are named 'Bus Stop' and 'Route Number'
    # Group by 'Route Number' and maintain the order
    route_dict = df.groupby("Route Number")["Bus Stop"].apply(list)

    # Convert to a list of lists
    route_list = route_dict.tolist()

    # Print the result
    return route_list



folder_path = r"C:\Users\sudha\NN_trainings\project 1\dataset"
excel_files = glob(os.path.join(folder_path, "*.xlsx"))

# Define filters
filters = {
    'YEAR': ["first"], # 2nd and 3rd year students
    'COLLEGE': ['SSN']  # SSN, SNU, Faculty
}

if __name__ == "__main__":
    import os
    from glob import glob

    # A function which generates a dictionary of location and corresponding coordinates
    coordinates_file_path = r"C:\Users\sudha\Downloads\bus_route_optimization_package\project 1\dataset\allRoutesLatLong.xlsx"
    coordinates_dict = create_location_coordinates(coordinates_file_path)
    print(coordinates_dict)

    # A function which generates list of routes
    route_list_file_path = r"C:\Users\sudha\Downloads\bus_route_optimization_package\project 1\dataset\routesList.xlsx"
    route_list = create_routes_list(route_list_file_path)
    print(route_list)

    demand_distribution = create_demand_distribution(excel_files, filters)
    print(demand_distribution)

{'Ambattur Estate': (13.10051, 80.1637), 'Ratinakanaru': (12.6842, 79.98333), 'Chengalpettu New BS': (12.69136, 79.98064), 'Chengalpettu Old BS': (12.69715, 79.97664), 'Mahindra City': (12.74371, 79.99251), 'Singaperumal Koil Signal': (12.76168, 80.00351), 'Ford BS': (12.78727, 80.015), 'Maraimalai Nagar BS': (12.79982, 80.0233), 'HP PB': (12.80286, 80.02538), 'Gurukulam': (12.815, 80.03268), 'Potheri BS': (12.82079, 80.03728), 'A2Z': (12.83714, 80.05204), 'Mambakkam': (12.82805, 80.16537), 'Peravallur BS': (13.11675, 80.23088), 'Venus (Gandhi Statue)': (13.11179, 80.239), 'Perambur Rly St.': (13.10861, 80.24235), 'Jamalia': (13.10692, 80.2476), 'Ottery': (13.09722, 80.25122), 'Porur (Kumar Sweets)': (13.03482, 80.15599), 'Saravana stores (shell PB)': (13.02178, 80.14535), 'Mugalaivakkam BS': (13.02128, 80.16065), 'Ramapuram BS': (13.02408, 80.17839), 'Sanitorium (GK Hotel)': (12.93815, 80.12928081), 'Perungalathur': (12.90655, 80.09684), 'Beach Station': (13.09302, 80.29224), 'MGR Jan

In [22]:
!pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)


In [63]:

# optimize_routes.py
import pandas as pd
from ortools.sat.python import cp_model
from math import radians, sin, cos, sqrt, atan2
from itertools import combinations
import os

# Settings
CAPACITY = 55
FILTERS = {
    'YEAR': ['2', '4'],
    'COLLEGE': ['SSN']
}
DATASET_DIR = R"C:\Users\sudha\NN_trainings\project 1\dataset"

# Haversine distance function
def haversine_distance(coord1, coord2):
    lat1, lon1 = map(radians, coord1)
    lat2, lon2 = map(radians, coord2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return 6371 * c

# Load demand
def create_demand_distribution():
    files = [f for f in os.listdir(DATASET_DIR) if f.startswith("R") and f.endswith(".xlsx")]
    all_data = pd.DataFrame()
    for file in files:
        df = pd.read_excel(os.path.join(DATASET_DIR, file))
        if all(col in df.columns for col in ["YEAR", "COLLEGE", "BUSTOP"]):
            filtered = df[
                df["YEAR"].str.lower().isin(FILTERS["YEAR"]) &
                df["COLLEGE"].isin(FILTERS["COLLEGE"])
            ]
            all_data = pd.concat([all_data, filtered], ignore_index=True)
    return all_data.groupby("BUSTOP").size().to_dict()

# Load coordinates
def load_coordinates():
    df = pd.read_excel(os.path.join(DATASET_DIR, "allRoutesLatLong.xlsx"))
    coords_df = df.set_index("Location")[["Latitude", "Longitude"]].dropna()
    return coords_df.apply(tuple, axis=1).to_dict()

# Load routes
def load_routes():
    df = pd.read_excel(os.path.join(DATASET_DIR, "routesList.xlsx"))
    route_dict = df.groupby("Route Number")["Bus Stop"].apply(list)
    return route_dict.tolist()

# OR-Tools CP-SAT Optimization
def optimize_bus_routes_cpsat(routes, passenger_demand, coordinates_dict, capacity=55):
    model = cp_model.CpModel()
    
    # Create a set of stops that appear in at least one route
    covered_stops = set()
    for route in routes:
        covered_stops.update(route)
    
    # Filter demand to only include covered stops
    feasible_demand = {stop: demand for stop, demand in passenger_demand.items() 
                      if stop in covered_stops}
    
    print(f"Note: Optimization will only consider {len(feasible_demand)} out of {len(passenger_demand)} stops with demand.")
    
    x = {r: model.NewBoolVar(f'x_{r}') for r in range(len(routes))}
    
    # Only apply constraints for stops that exist in at least one route
    for stop, demand in feasible_demand.items():
        model.Add(sum(x[r] for r in range(len(routes)) if stop in routes[r]) >= 1)
    
    for r in range(len(routes)):
        # Only consider demand for stops that are in feasible_demand
        total = sum(feasible_demand.get(stop, 0) for stop in routes[r])
        model.Add(x[r] * total <= capacity)
    
    model.Minimize(sum(x[r] for r in range(len(routes))))
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    result = []
    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        for r in range(len(routes)):
            if solver.Value(x[r]) > 0.5:
                route = routes[r]
                total = sum(passenger_demand.get(stop, 0) for stop in route)
                result.append((r, route, total))
    return result

def main():
    demand = create_demand_distribution()
    coords = load_coordinates()
    routes = load_routes()

    filtered_routes = [r for r in routes if all(s in coords for s in r)]
    filtered_demand = {k: v for k, v in demand.items() if k in coords}

    selected_routes = optimize_bus_routes_cpsat(filtered_routes, filtered_demand, coords, CAPACITY)
    for i, route, total in selected_routes:
        print(f"Bus {i+1}: {' -> '.join(route)} | Total students: {total}")

if __name__ == "__main__":
    main()

AttributeError: Can only use .str accessor with string values!

In [88]:
import pandas as pd
from ortools.sat.python import cp_model
from math import radians, sin, cos, sqrt, atan2
import os

# Settings
CAPACITY = 70
FILTERS = {
    'YEAR': ['1'],
    'COLLEGE': ['SSN']
}
DATASET_DIR = r"C:\Users\sudha\NN_trainings\project 1\dataset"

# Haversine distance function
def haversine_distance(coord1, coord2):
    lat1, lon1 = map(radians, coord1)
    lat2, lon2 = map(radians, coord2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return 6371 * c

# Load demand
def create_demand_distribution():
    files = [f for f in os.listdir(DATASET_DIR) if f.endswith(".xlsx")]
    all_data = pd.DataFrame()
    for file in files:
        df = pd.read_excel(os.path.join(DATASET_DIR, file))
        if all(col in df.columns for col in ["YEAR", "COLLEGE", "BUSTOP"]):
            filtered = df[
                df["YEAR"].astype(str).str.lower().isin(FILTERS["YEAR"]) &
                df["COLLEGE"].isin(FILTERS["COLLEGE"])
            ]
            all_data = pd.concat([all_data, filtered], ignore_index=True)
    return all_data.groupby("BUSTOP").size().to_dict()

# Load coordinates
def load_coordinates():
    df = pd.read_excel(r"C:\Users\sudha\NN_trainings\project 1\dataset\allLatsLongs1.xlsx")
    coords_df = df.set_index("Location")[["Latitude", "Longitude"]].dropna()
    return coords_df.apply(tuple, axis=1).to_dict()

# Load routes
def load_routes():
    df = pd.read_excel(r"C:\Users\sudha\NN_trainings\project 1\dataset\rl.xlsx")
    route_dict = df.groupby("Route Number")["Bus Stop"].apply(list)
    return route_dict.tolist()

# OR-Tools CP-SAT Optimization
def optimize_bus_routes_cpsat(routes, passenger_demand, coordinates_dict, capacity=55):
    model = cp_model.CpModel()
    x = {r: model.NewBoolVar(f'x_{{r}}') for r in range(len(routes))}

    for stop, demand in passenger_demand.items():
        model.Add(sum(x[r] for r in range(len(routes)) if stop in routes[r]) >= 1)

    for r in range(len(routes)):
        total = sum(passenger_demand.get(stop, 0) for stop in routes[r])
        model.Add(x[r] * total <= capacity)

    model.Minimize(sum(x[r] for r in range(len(routes))))
    solver = cp_model.CpSolver()
    status = solver.Solve(model)
    print("Solver status:", solver.StatusName(status))

    result = []
    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        for r in range(len(routes)):
            if solver.Value(x[r]) > 0.5:
                route = routes[r]
                total = sum(passenger_demand.get(stop, 0) for stop in route)
                result.append((r, route, total))
    return result

demand = create_demand_distribution()
coords = load_coordinates()    
routes = load_routes()

filtered_routes = [r for r in routes if all(s in coords for s in r)]
filtered_demand = {k: v for k, v in demand.items() if k in coords}

selected_routes = optimize_bus_routes_cpsat(filtered_routes, filtered_demand, coords, CAPACITY)
print(selected_routes)
for i, route, total in selected_routes:
    print(f"Bus {i+1}: {' -> '.join(route)} | Total students: {total}")

Solver status: INFEASIBLE
[]


In [89]:
# Modify your main code to include this before optimization
stops_in_demand = set(filtered_demand.keys())
stops_in_routes = set()
for route in filtered_routes:
    stops_in_routes.update(route)

uncovered_stops = stops_in_demand - stops_in_routes
if uncovered_stops:
    print(f"Warning: {len(uncovered_stops)} stops in demand are not covered by any route:")
    print(uncovered_stops)

# Filter demand to only include stops that exist in at least one route
filtered_demand = {k: v for k, v in filtered_demand.items() if k in stops_in_routes}
print(f"Filtered demand to {len(filtered_demand)} stops that exist in routes.")

# Then run your existing optimization
selected_routes = optimize_bus_routes_cpsat(filtered_routes, filtered_demand, coords, CAPACITY)

{'Perungalathur'}
Filtered demand to 114 stops that exist in routes.
Solver status: OPTIMAL


In [90]:
for i, route, total in selected_routes:
    print(f"Bus {i+1}: {' -> '.join(route)} | Total students: {total}")

Bus 1: Ambattur Estate -> College | Total students: 5
Bus 2: Ashok Pillar -> CMBT Park -> CPWD Quarters -> Ekattuthangal(AlagappaU) -> Goldmine Hotel -> Jeyachandra Pallikaranai -> Kasi Theatre(Hondai Showroom) -> MMDA(valluva H) -> Sri Sivasubramaniya Nadar College of Engineering -> Thirumangalam(opp to VR Mahal) -> Tirunagar (Periyar Pathai) | Total students: 6
Bus 3: Anna Arivalayam -> College -> Dhasprakash -> Egmore stadium -> IITM -> Jamalia -> Palavakkam(opp.Govt school) -> Perambur BS -> Perambur RS -> SIET College -> kanathur -> kovil -> perambur market | Total students: 6
Bus 4: Chetpet signal -> EGA -> ESI -> ICF -> Iynavaram Rly Qtrs -> Joint Office -> Kellys -> Kotturpuram -> Madhya Kailash Temple -> Railway Station Indira Nagar -> Sayani -> Shastri Bhavan
(College Road) -> Sterling Road Junction -> Villivakkam (Shanthi Bhavan) -> college -> kilpauk garden | Total students: 2
Bus 5: Alwarpet Narathagana Sabha -> Boojanvilla Jn -> Chetput Signal -> Chintamani -> Jain Colleg

In [91]:
len(selected_routes)

41